# Notebook Used to Generate Benchmark Results 

In [1]:
import polars as pl
import polars_ds as pds

# Parallel ML metrics evaluations on segments 

Use cases:

1. Evaluate ML model performance in market A, B, C.
2. The Dataframe contains a column that defines the "split" of the dataframe. Then this can simulatneously evaluate ML model's performances on each of the train, test, recent, or any other split you have.
3. Evaluate ML model performance over time, e.g. weekly / monthly 

Comparison: 

Polars + PDS vs Pandas + Sklearn

In [2]:
# Generate a 
from datetime import date

dates = pl.date_range(date(2020, 1, 1), date(2024, 10, 1), "1d", eager=True)
df = pds.frame(size=len(dates)).select(
    pds.random().alias("predicted"),
    (pds.random() > 0.25).cast(pl.UInt8).alias("actual_target"),
    dates = dates,
)
df_pd = df.to_pandas()

In [3]:
df_pd

,predicted,actual_target,dates
0,0.413767,1,2020-01-01
1,0.125783,1,2020-01-02
2,0.382943,1,2020-01-03
3,0.690455,0,2020-01-04
4,0.492488,0,2020-01-05
...,...,...,...
1731,0.365318,1,2024-09-27
1732,0.635105,1,2024-09-28
1733,0.156054,1,2024-09-29
1734,0.736704,1,2024-09-30


In [4]:
import pandas as pd
from sklearn.metrics import roc_auc_score, log_loss, brier_score_loss

In [5]:
%%timeit
df_pd["year"] = df['dates'].dt.year()
df_pd.groupby(["year"]).apply(
    lambda df_group: pd.Series({
        "count": len(df_group["actual_target"]),
        "roc_auc": roc_auc_score(df_group["actual_target"], df_group["predicted"]),
        "log_loss": roc_auc_score(df_group["actual_target"], df_group["predicted"])
    })
    , include_groups=False
)

10.3 ms ± 83 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit
df.group_by(pl.col("dates").dt.year()).agg(
    count = pl.len(),
    roc_auc = pds.query_roc_auc("actual_target", "predicted"),
    log_loss = pds.query_log_loss("actual_target", "predicted")
).sort("dates")
# 1/5 of the time, less lines of code + easier to understand syntax

2.13 ms ± 67.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
